In [164]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import polars as pl
from scipy.optimize import minimize
from sklearn.linear_model import LinearRegression

In [47]:
N = 10 # control units
T = 10 # observed periods
T0 = 5 # intervention time

# Y(0) ~ N(0, 1)
# Y(1) ~ N(4, 1)
np.random.seed(20240209)
weight = np.array(
    [0.4, 0.0, 0.3, 0.0, 0.0, 0.0, 0.3, 0.0, 0.0, 0.0]
)
Y_control = np.random.normal(0, 1, size=(N, T))
Y0 = weight @ Y_control
Y0_observed = np.concatenate(
    [
        Y0[:T0], np.random.normal(5, 1, T-T0)
    ]
)
Y0_counterfactual = Y0


fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=np.arange(1, T+1),
        y=Y0_observed,
        mode='lines',
        name='observed'
    )
)
fig.add_trace(
    go.Scatter(
        x=np.arange(T0, T+1),
        y=Y0_counterfactual[T0-1:],
        name='counterfactual',
        line=dict(
            dash='dash'
        ),
        mode='lines'
    )
)
for i in range(N):
    if i==1:
        fig.add_trace(
            go.Scatter(
                x=np.arange(1, T+1),
                y=Y_control[i, :],
                name='control units',
                opacity=0.3,
                line=dict(
                    color='gray'
                ),
                mode='lines',
            )
        )
    else:
        fig.add_trace(
            go.Scatter(
                x=np.arange(1, T+1),
                y=Y_control[i, :],
                opacity=0.3,
                line=dict(
                    color='gray'
                ),
                mode='lines',
                showlegend=False
            )
        )
fig.add_vline(
    x=5
)
fig.update_layout(
    xaxis_title='time',
    yaxis_title='outcome'
)
fig.show()

In [192]:
# stataファイルを読み込む
df = pd.read_stata('./data/smoking.dta')
df = pl.from_pandas(df)
# print(df.head().to_pandas().to_markdown())
df = df.select(
    "state", "year", "cigsale", "retprice"
).pivot(
    values=['cigsale', 'retprice'],
    index='year',
    columns='state'
)
# 処置群とコントロール群に分割
df

year,cigsale_state_Alabama,cigsale_state_Arkansas,cigsale_state_California,cigsale_state_Colorado,cigsale_state_Connecticut,cigsale_state_Delaware,cigsale_state_Georgia,cigsale_state_Idaho,cigsale_state_Illinois,cigsale_state_Indiana,cigsale_state_Iowa,cigsale_state_Kansas,cigsale_state_Kentucky,cigsale_state_Louisiana,cigsale_state_Maine,cigsale_state_Minnesota,cigsale_state_Mississippi,cigsale_state_Missouri,cigsale_state_Montana,cigsale_state_Nebraska,cigsale_state_Nevada,cigsale_state_New Hampshire,cigsale_state_New Mexico,cigsale_state_North Carolina,cigsale_state_North Dakota,cigsale_state_Ohio,cigsale_state_Oklahoma,cigsale_state_Pennsylvania,cigsale_state_Rhode Island,cigsale_state_South Carolina,cigsale_state_South Dakota,cigsale_state_Tennessee,cigsale_state_Texas,cigsale_state_Utah,cigsale_state_Vermont,cigsale_state_Virginia,…,retprice_state_California,retprice_state_Colorado,retprice_state_Connecticut,retprice_state_Delaware,retprice_state_Georgia,retprice_state_Idaho,retprice_state_Illinois,retprice_state_Indiana,retprice_state_Iowa,retprice_state_Kansas,retprice_state_Kentucky,retprice_state_Louisiana,retprice_state_Maine,retprice_state_Minnesota,retprice_state_Mississippi,retprice_state_Missouri,retprice_state_Montana,retprice_state_Nebraska,retprice_state_Nevada,retprice_state_New Hampshire,retprice_state_New Mexico,retprice_state_North Carolina,retprice_state_North Dakota,retprice_state_Ohio,retprice_state_Oklahoma,retprice_state_Pennsylvania,retprice_state_Rhode Island,retprice_state_South Carolina,retprice_state_South Dakota,retprice_state_Tennessee,retprice_state_Texas,retprice_state_Utah,retprice_state_Vermont,retprice_state_Virginia,retprice_state_West Virginia,retprice_state_Wisconsin,retprice_state_Wyoming
f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
1970.0,89.800003,100.300003,123.0,124.800003,120.0,155.0,109.900002,102.400002,124.800003,134.600006,108.5,114.0,155.800003,115.900002,128.5,104.300003,93.400002,121.300003,111.199997,108.099998,189.5,265.700012,90.0,172.399994,93.800003,121.599998,108.400002,107.300003,123.900002,103.599998,92.699997,99.800003,106.400002,65.5,122.599998,124.300003,…,38.799999,29.4,42.200001,39.0,34.299999,33.799999,41.400002,30.6,37.700001,34.200001,28.299999,34.299999,38.0,39.099998,36.200001,36.0,34.0,33.900002,38.900002,31.4,39.700001,27.299999,37.299999,36.599998,38.400002,38.400002,39.299999,32.5,38.5,39.900002,40.400002,34.599998,37.700001,28.799999,33.700001,38.5,34.099998
1971.0,95.400002,104.099998,121.0,125.5,117.599998,161.100006,115.699997,108.5,125.599998,139.300003,108.400002,102.800003,163.5,119.800003,133.199997,116.400002,105.400002,127.599998,115.599998,108.599998,190.5,278.0,92.599998,187.600006,98.5,124.599998,115.400002,106.300003,123.199997,115.0,96.699997,106.300003,108.900002,67.699997,124.400002,128.399994,…,39.700001,31.1,45.5,41.299999,35.799999,33.599998,41.400002,32.200001,38.5,38.900002,30.1,39.299999,38.799999,40.099998,37.5,36.799999,34.700001,34.700001,44.0,34.099998,41.700001,29.4,38.900002,38.099998,39.799999,44.700001,40.200001,34.299999,38.5,41.599998,42.0,36.599998,39.5,30.200001,41.599998,40.200001,34.400002
1972.0,101.099998,103.900002,123.5,134.300003,110.800003,156.300003,117.0,126.099998,126.599998,149.199997,109.400002,111.0,179.399994,125.300003,136.5,96.800003,112.099998,130.0,122.199997,104.900002,198.600006,296.200012,99.300003,214.100006,103.800003,124.400002,121.699997,109.0,134.399994,118.699997,103.0,111.5,108.599998,71.300003,138.0,137.0,…,39.900002,31.200001,51.299999,44.700001,40.900002,33.700001,41.900002,32.5,41.900002,38.799999,30.6,40.0,41.5,45.200001,37.400002,37.700001,40.099998,41.099998,40.599998,36.099998,41.099998,28.700001,38.900002,38.400002,39.799999,44.700

In [307]:
df = pd.read_stata('./data/smoking.dta')
df = pl.from_pandas(df)

T = len(df['year'].unique()) # 観測された時間の長さ
N = len(df['state'].unique()) - 1 # control unitの数
T0 = 1988 - 1970 + 1 # 処置が始まるまでの時間

# print(df.head().to_pandas().to_markdown())
df = df.select(
    "state", "year", "cigsale", "retprice"
)

# X: N×T0*2 matrix
X = df.filter(
(pl.col("state") != "California")
& (pl.col("year") <= 1988)
).sort('year')[['cigsale', 'retprice']].to_numpy().T.reshape(N, -1)

Y = df.filter(
    (pl.col("state") == "California")
    & (pl.col("year") <= 1988)
)[['cigsale', 'retprice']].to_numpy().T.reshape(2*T0)

print(X.shape)

(38, 38)


In [367]:
# OLS
alpha_OLS = np.linalg.inv(X.T @ X) @ X.T @ Y
print(alpha_OLS.round(3))
print(alpha_OLS.sum())

[-0.618 -0.918  0.633  0.276  0.263  0.958  0.168  0.303 -0.331 -0.099
 -0.578  0.957 -0.356  0.278 -0.03  -0.455 -0.164 -0.621 -0.175 -0.077
  0.079  0.382 -0.326  0.419  0.216 -0.125 -0.1    0.21  -0.398  0.734
  0.315 -0.058  0.504 -0.21  -0.767  0.737  0.026 -0.099]
0.9510519


In [336]:
treatment_outcome = df.filter(
    (pl.col("state") == "California")
)[['cigsale']].to_numpy().T.reshape(T)

control_outcome = df.filter(
    (pl.col("state") != "California")
)[['cigsale']].to_numpy().reshape(N, -1).T

print(control_outcome.shape)

pred_OLS = control_outcome @ alpha_OLS

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=year,
        y=treatment_outcome,
        name='California'
    )
)

fig.add_trace(
    go.Scatter(
        x=year,
        y=pred_OLS,
        name='Synthetic California'
    )
)


fig.add_vline(
    x=1988
)

fig.update_layout(
    template='plotly_white',
    xaxis_title='year',
    yaxis_title='cigarette sales (in packs)',
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
    ),
    width=1200,
    height=600,
)

(31, 38)


In [368]:
# SCM
def loss_function(w, X, y):
    return np.mean((y - X @ w)**2)

cons = (
    {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}
)
bounds = [(0, None) for i in range(N)]

alpha0 = np.ones(N) / N
scm = minimize(
    loss_function,
    x0=alpha0,
    args=(X, Y),
    constraints=cons,
    bounds=bounds,
    method="SLSQP"
)
alpha_scm = scm.x
print(alpha_scm.round(4))
print(alpha_scm.sum())

[0.     0.     0.     0.0853 0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.113
 0.1051 0.4566 0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.2401 0.     0.     0.     0.     0.    ]
1.000000000008864


In [335]:
pred_scm = control_outcome @ alpha_scm

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=year,
        y=treatment_outcome,
        name='California'
    )
)

fig.add_trace(
    go.Scatter(
        x=year,
        y=pred_scm,
        name='Synthetic California'
    )
)

fig.add_trace(
    go.Scatter(
        x=year,
        y=pred_OLS,
        name='OLS',
        opacity=0.5
    )
)

fig.add_vline(
    x=1988
)

fig.update_layout(
    template='plotly_white',
    xaxis_title='year',
    yaxis_title='cigarette sales (in packs)',
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
    ),
    width=1200,
    height=600,
)

In [339]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=year,
        y=treatment_outcome - pred_scm,
        name='SCM'
    )
)

fig.add_trace(
    go.Scatter(
        x=year,
        y=treatment_outcome - pred_OLS,
        opacity=0.5,
        name='OLS'
    )
)

fig.add_vline(
    x=1988
)

fig.add_hline(
    y=0,
    opacity=0.3
)

fig.update_layout(
    template='plotly_white',
    xaxis_title='year',
    yaxis_title='Treatment Effect',
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
    ),
    width=1200,
    height=600,
    title='Estimatied Result of Treatment Effect'
)

fig.show()

In [350]:
state = df.filter(pl.col("state")!="California")['state'].unique()
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=state,
        y=alpha_scm,
        name="SCM"
    )
)

fig.add_trace(
    go.Bar(
        x=state,
        y=alpha_OLS,
        name="OLS"
    )
)
fig.update_layout(
    template='plotly_white',
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
    ),
    xaxis_title="state",
    yaxis_title="weight",
    xaxis_tickangle=270,
    width=1200,
    height=600,
)

In [364]:
state_used = ["Connecticut", "Nevada", "New Hampshire", "New Mexico", "Utah"]
# df.filter(pl.col('state').is_in(state_used))
fig = go.Figure()
fig.add_trace(
        go.Scatter(
            x=year,
            y=df.filter(pl.col("state")=="California")['cigsale'],
            name=c
        )
    )
for c in state_used:
    fig.add_trace(
        go.Scatter(
            x=year,
            y=df.filter(pl.col("state")==c)['cigsale'],
            name=c,
            # line=dict({"color":'gray'}),
            opacity=0.5
        )
    )

fig.update_layout(
    template='plotly_white',
    xaxis_title='year',
    yaxis_title='cigarette sales (in packs)',
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
    ),
    width=1200,
    height=600,
)
fig.show()